In [2]:
%load_ext autoreload
%autoreload 2

# Neo4j v2.0.1

**Descarga**
- [Descargar Neo4j Desktop v2.0.1 (macOS)](https://neo4j.com/download/neo4j-desktop/?edition=desktop&flavour=osx&release=2.0.1&offline=false)

**Configuración**
1. Instalar Neo4j Desktop.
2. Abrir la app y crear un nuevo proyecto con "Add".
3. Dentro del proyecto, seleccionar "Add Graph" → "Local DBMS".
4. Configurar:
   - Name: `BDNR_ml-25m`
   - Password: `bdnr2025`
   - Versión usada: `2025.05.0`
5. Hacer clic en "Create".

**Conexión**
```
neo4j://127.0.0.1:7687
```

In [46]:
import json
from src.benchmark.utils import get_system_info
import neo4j

print("neo4j Version:", neo4j.__version__)
print("System Info:\n", json.dumps(get_system_info(), indent=3))

neo4j Version: 5.28.1
System Info:
 {
   "python_version": "3.12.1",
   "system": {
      "os": "Darwin",
      "release": "24.5.0",
      "machine": "x86_64",
      "processor": "i386"
   },
   "cpu": {
      "physical_cores": 8,
      "total_cores": 8,
      "frequency_mhz": 2400
   },
   "memory": {
      "total_ram_gb": 16.0,
      "available_ram_gb": 0.75
   }
}


# Data Load

In [48]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"  
user = "neo4j"
password = "bdnr2025" 

driver = GraphDatabase.driver(uri, auth=(user, password))
session = driver.session()

In [51]:
try:
    session.run("RETURN 1").single()
    print("Successfully connected to Neo4j")
except Exception as e:
    print(f"Connection failed: {e}")

Successfully connected to Neo4j


In [52]:
databases = session.run("SHOW DATABASES").data()
print("Databases:", [db["name"] for db in databases])

Databases: ['neo4j', 'system']


In [53]:
from pprint import pprint

result = session.run("SHOW DATABASES").data()
for db in result:
    print(f"\n{db['name']}")
    pprint(db, indent=2, width=100, depth=3, sort_dicts=False)


neo4j
{ 'name': 'neo4j',
  'type': 'standard',
  'aliases': [],
  'access': 'read-write',
  'address': 'localhost:7687',
  'role': 'primary',
  'writer': True,
  'requestedStatus': 'online',
  'currentStatus': 'online',
  'statusMessage': '',
  'default': True,
  'home': True,
  'constituents': []}

system
{ 'name': 'system',
  'type': 'system',
  'aliases': [],
  'access': 'read-write',
  'address': 'localhost:7687',
  'role': 'primary',
  'writer': True,
  'requestedStatus': 'online',
  'currentStatus': 'online',
  'statusMessage': '',
  'default': False,
  'home': False,
  'constituents': []}


### Batch Load: UNWIND & MERGE

In [8]:
import pandas as pd

# Read CSV files
base_path = "/Users/efaliveni//Downloads/ml-25m"

ratings = pd.read_csv(f"{base_path}/ratings.csv")
movies = pd.read_csv(f"{base_path}/movies.csv")
tags = pd.read_csv(f"{base_path}/tags.csv")
genome_tags = pd.read_csv(f"{base_path}/genome-tags.csv")
genome_scores = pd.read_csv(f"{base_path}/genome-scores.csv")

In [9]:
# Filter Data
SAMPLE_SIZE = 500

selected_users = ratings['userId'].unique()[:SAMPLE_SIZE]
filtered_ratings = ratings[ratings['userId'].isin(selected_users)]
filtered_movie_ids = filtered_ratings['movieId'].unique()
filtered_movies = movies[movies['movieId'].isin(filtered_movie_ids)].copy()
filtered_tags = tags[tags['movieId'].isin(filtered_movie_ids)]
filtered_scores = genome_scores[genome_scores['movieId'].isin(filtered_movie_ids)]

In [10]:
from collections import defaultdict
# Create a dictionary of genome tags for quick lookup
genome_tags_dict = {row['tagId']: row['tag'] for _, row in genome_tags.iterrows()}

# Build dictionaries to accumulate data
ratings_by_movie = defaultdict(list)
scores_by_movie = defaultdict(list)

ratings_by_user = defaultdict(list)
tags_by_user = defaultdict(list)

In [11]:
# Process ratings and tags to build both movie and user views
RELEVANCE_THRESHOLD = 0.8

# Ratings
for _, r in filtered_ratings.iterrows():
    movie_id = int(r.movieId)
    user_id = int(r.userId)
    
    # Add to movie's ratings
    ratings_by_movie[movie_id].append({
        "userId": user_id,
        "rating": float(r.rating),
        "timestamp": int(r.timestamp)
    })
    
    # Add to user's ratings
    ratings_by_user[user_id].append({
        "movieId": movie_id,
        "rating": float(r.rating),
        "timestamp": int(r.timestamp)
    })
    
# Tags (only for users), genome tag is the cleaned infered tag
for _, t in filtered_tags.dropna(subset=["tag"]).iterrows():
    movie_id = int(t.movieId)
    user_id = int(t.userId)
    
    tags_by_user[user_id].append({
        "movieId": movie_id,
        "tag": t.tag,
        "timestamp": int(t.timestamp)
    })
    
# Genome scores (only for movies), with 0.8 threshold
for _, s in filtered_scores[filtered_scores.relevance > RELEVANCE_THRESHOLD].iterrows():
    movie_id = int(s.movieId)
    tag_id = int(s.tagId)
    
    scores_by_movie[movie_id].append({
        "tagId": tag_id,
        "tag": genome_tags_dict.get(tag_id, ""),
        "relevance": float(s.relevance)
    })

In [12]:
import re

# Build movie documents
movie_docs = []
for _, m in filtered_movies.iterrows():
    movie_id = int(m.movieId)
   
    # Extract year from title
    title = m.title
    year_match = re.search(r"\((\d{4})\)$", title)
    year = int(year_match.group(1)) if year_match else None
    
    # Get ratings and tags for this movie
    movie_ratings = ratings_by_movie.get(movie_id, [])
    tag_entries = scores_by_movie.get(movie_id, [])
    
    # Calculate statistics
    rating_count = len(movie_ratings)
    if rating_count > 0:
        avg_rating = sum(r['rating'] for r in movie_ratings) / rating_count
        # Calculate rating distribution (count of each star rating)
        rating_dist = {"0.5":0, "1.0":0, "1.5":0, "2.0":0, "2.5":0, 
                      "3.0":0, "3.5":0, "4.0":0, "4.5":0, "5.0":0}
        for r in movie_ratings:
            rating_key = f"{r['rating']}"
            rating_dist[rating_key] += 1 
    else:
        avg_rating = 0.0
        rating_dist = {}  # will result in empty arrays below

    # Neo4j only allows:
    # - Single values: string, int, float, boolean, datetime
    # - Arrays of primitive types: [int], [string], etc.
    dist_keys = list(rating_dist.keys()) if rating_dist else []
    dist_values = list(rating_dist.values()) if rating_dist else []
    
    tag_ids = [int(e["tagId"]) for e in tag_entries]
    tag_relevances = [float(e["relevance"]) for e in tag_entries]
    tag_names = [str(e["tag"]) for e in tag_entries]
    
    movie_docs.append({
        "movieId": movie_id,
        "title": title,
        "year": year,
        "genres": m.genres if isinstance(m.genres, list) else [],
        "ratingCount": rating_count,
        "avgRating": avg_rating,
        "ratingDistKeys": dist_keys,
        "ratingDistValues": dist_values,
        "tagIds": tag_ids,
        "tagRelevances": tag_relevances,
        "tagNames": tag_names
    })

In [13]:
user_docs = []

for user_id in selected_users:
    user_id = int(user_id)

    user_ratings = ratings_by_user.get(user_id, [])
    user_tags = tags_by_user.get(user_id, [])
    
    # Rating stats
    rating_count = len(user_ratings)
    avg_rating = sum(r['rating'] for r in user_ratings) / rating_count if rating_count > 0 else 0.0
    
    # Sort ratings by timestamp
    user_ratings.sort(key=lambda r: r['timestamp'])

    # Flatten ratings for Neo4j
    rated_movie_ids = [r['movieId'] for r in user_ratings]
    rated_scores = [r['rating'] for r in user_ratings]
    rated_timestamps = [r['timestamp'] for r in user_ratings]

    # (Optional future support for tags)
    # tag_movie_ids = [t['movieId'] for t in user_tags]
    # tag_labels = [t['tag'] for t in user_tags]
    # tag_timestamps = [t['timestamp'] for t in user_tags]

    user_docs.append({
        "userId": user_id,
        "ratingCount": rating_count,
        "avgRating": round(avg_rating, 2),
        "ratedMovieIds": rated_movie_ids,
        "ratedScores": rated_scores,
        "ratedTimestamps": rated_timestamps,
        # "tagMovieIds": tag_movie_ids,
        # "tagLabels": tag_labels,
        # "tagTimestamps": tag_timestamps
    })


In [55]:
from neo4j import Session

def load_users(session: Session, user_data: list, batch_size: int = 1000) -> None:
    """Creates User nodes with rating history embedded as arrays."""
    
    query = """
    UNWIND $batch AS user
    MERGE (u:User {userId: user.userId})
    SET u.ratingCount = user.ratingCount,
        u.avgRating = user.avgRating,
        u.ratedMovieIds = user.ratedMovieIds,
        u.ratedScores = user.ratedScores,
        u.ratedTimestamps = user.ratedTimestamps
    """

    if batch_size is None:
        batch_size = len(user_data)

    for i in range(0, len(user_data), batch_size):
        batch = user_data[i:i + batch_size]
        session.run(query, batch=batch)


def load_movies(session: Session, movie_data: list, batch_size: int = 1000) -> None:
    """Creates Movie nodes with embedded genome tags (tagGenome)."""
         
    # Cypher query to store Movie nodes with embedded data
    query = """
    UNWIND $batch AS movie
    MERGE (m:Movie {movieId: movie.movieId})
    SET m.title = movie.title,
        m.year = movie.year,
        m.genres = movie.genres,
        m.ratingCount = movie.ratingCount,
        m.avgRating = movie.avgRating,
        m.ratingDistKeys = movie.ratingDistKeys,
        m.ratingDistValues = movie.ratingDistValues,        
        m.tagIds = movie.tagIds,
        m.tagRelevances = movie.tagRelevances,
        m.tagNames = movie.tagNames
    """
    
    if batch_size is None:
        batch_size = len(movie_data)

    for i in range(0, len(movie_data), batch_size):
        batch = movie_data[i:i + batch_size]
        session.run(query, batch=batch)


def load_ratings(session: Session, ratings_df: pd.DataFrame, batch_size: int = 1000) -> None:
    """Creates User and Movie nodes with RATED relationships."""
    rating_data = ratings_df.to_dict("records")
    
    query = """
    UNWIND $batch AS row
    MERGE (u:User {userId: row.userId})
    MERGE (m:Movie {movieId: row.movieId})
    MERGE (u)-[r:RATED]->(m)
    SET r.rating = row.rating, 
        r.timestamp = datetime({epochSeconds: row.timestamp})
    """
   
    if batch_size is None:
        batch_size = len(ratings_df)
    
    # Process in batches to avoid memory issues
    for i in range(0, len(rating_data), batch_size):
        batch = rating_data[i:i + batch_size]
        session.run(query, batch=batch)

def load_tags(session: Session, tags_df: pd.DataFrame, batch_size: int = 1000) -> None:
    """Creates TAGGED relationships between existing Users and Movies"""
    # Clean data - remove null tags
    tag_data = tags_df.dropna(subset=["tag"]).to_dict("records")
    
    query = """
    UNWIND $batch AS row
    MATCH (u:User {userId: row.userId})
    MATCH (m:Movie {movieId: row.movieId})
    MERGE (u)-[t:TAGGED]->(m)
    SET t.tag = row.tag
    """
       
    if batch_size is None:
        batch_size = len(tags_df)
        
    for i in range(0, len(tag_data), batch_size):
        batch = tag_data[i:i + batch_size]
        session.run(query, batch=batch)

In [56]:
result = session.run("MATCH (n) DETACH DELETE n")
summary = result.consume()

print("Query summary:")
print(f"Nodes deleted: {summary.counters.nodes_deleted}")
print(f"Relationships deleted: {summary.counters.relationships_deleted}")

Query summary:
Nodes deleted: 0
Relationships deleted: 0


In [57]:
import time

start_time = time.time()

stats = {
    'movies_loaded': 0,
    'users_loaded': 0,
    'rates_created': 0,
    'tags_created': 0,
    'execution_time': 0
}

BATCH_SIZE = None

# Insert data
try:
    print("Starting data loading process...")
    
    # Load users (foundational nodes)
    print("\nLoading users...")
    user_start = time.time()
    load_users(session, user_docs, BATCH_SIZE)
    stats['users_loaded'] = len(user_docs)
    print(f"Users loaded in {time.time() - user_start:.2f}s")
    
    # Load movies (foundational nodes)
    print("\nLoading movies...")
    movie_start = time.time()
    load_movies(session, movie_docs, BATCH_SIZE)
    stats['movies_loaded'] = len(movie_docs)
    print(f"Movies loaded in {time.time() - movie_start:.2f}s")
    
    # Load users through ratings
    print("\nLoading ratings...")
    ratings_start = time.time()
    load_ratings(session, filtered_ratings, BATCH_SIZE)
    stats['rates_created'] = len(filtered_ratings)
    print(f"Ratings loaded in {time.time() - ratings_start:.2f}s")
    
    # Load tags
    print("\nLoading tags...")
    tags_start = time.time()
    load_tags(session, filtered_tags, BATCH_SIZE)
    stats['tags_created'] = len(filtered_tags)
    print(f"Tags loaded in {time.time() - tags_start:.2f}s")

except Exception as e:
    print(f"\nError during data loading: {str(e)}")
    raise
finally:
    stats['execution_time'] = time.time() - start_time
    print(f"\nTotal processing time: {stats['execution_time']:.2f} seconds")
    print(f"Summary: {stats}")

Starting data loading process...

Loading users...
Users loaded in 0.60s

Loading movies...
Movies loaded in 6.33s

Loading ratings...
Ratings loaded in 86.42s

Loading tags...
Tags loaded in 76.91s

Total processing time: 170.26 seconds
Summary: {'movies_loaded': 7141, 'users_loaded': 500, 'rates_created': 62834, 'tags_created': 811156, 'execution_time': 170.26220202445984}


### Sample Movie

In [63]:
query = """
MATCH (m:Movie)
RETURN m
LIMIT 1
"""

result = session.run(query)

for record in result:
    movie_node = record["m"]
    pprint(dict(movie_node), width=200, compact=True)

{'avgRating': 2.5,
 'genres': [],
 'movieId': 8977,
 'ratingCount': 6,
 'ratingDistKeys': ['0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '3.5', '4.0', '4.5', '5.0'],
 'ratingDistValues': [0, 0, 2, 0, 1, 2, 1, 0, 0, 0],
 'tagIds': [132, 359, 508, 509, 564],
 'tagNames': ['big budget', 'epic', 'historical', 'history', 'irish accent'],
 'tagRelevances': [0.8362499999999999, 0.92325, 0.9425, 0.89625, 0.8847499999999999],
 'title': 'Alexander (2004)',
 'year': 2004}


### Sample User

In [64]:
query = """
MATCH (u:User)
RETURN u
LIMIT 1
"""

result = session.run(query)

for record in result:
    user_node = record["u"]
    pprint(dict(user_node), width=200, compact=True) 

{'avgRating': 3.81,
 'ratedMovieIds': [5952, 2012, 2011, 1653, 1250, 6539, 6377, 3448, 1088, 899, 4308, 2161, 6711, 3949, 8360, 5878, 306, 1175, 307, 1237, 7327, 8154, 7234, 2843, 4144, 7365, 2068, 4422, 4973, 6016,
                   8873, 2692, 27721, 7323, 6954, 8014, 7939, 6370, 8973, 4703, 31956, 5147, 8786, 1260, 2351, 7940, 7209, 8685, 7820, 7937, 7938, 8405, 4325, 2632, 1217, 8729, 5912, 5767, 665, 2573,
                   27266, 8327, 32591, 5269, 3569, 27193, 5684, 7318, 296, 7361],
 'ratedScores': [4.0, 2.5, 2.5, 4.0, 4.0, 3.5, 4.0, 4.0, 4.0, 3.5, 3.0, 3.5, 5.0, 5.0, 4.0, 4.0, 3.5, 3.5, 5.0, 5.0, 3.5, 5.0, 4.5, 4.5, 5.0, 4.0, 2.5, 3.0, 4.5, 5.0, 3.0, 5.0, 3.0, 3.5, 3.5, 3.5,
                 2.5, 4.5, 4.0, 4.0, 3.5, 4.0, 4.0, 3.5, 4.5, 4.5, 4.0, 1.0, 2.5, 3.0, 2.5, 3.5, 5.0, 5.0, 3.5, 3.5, 3.0, 5.0, 5.0, 4.0, 4.5, 5.0, 5.0, 0.5, 5.0, 3.0, 2.0, 2.0, 5.0, 5.0],
 'ratedTimestamps': [1147868053, 1147868068, 1147868079, 1147868097, 1147868414, 1147868461, 1147868469, 1147868480, 

# Single-Threaded Benchmark
Simulate User Load, real time latency stress

In [68]:
from neo4j import Session
from typing import Union

def print_plan_tree(plan: Union[dict, object], indent=0):
    prefix = "  " * indent
    if isinstance(plan, dict):
        operator = plan.get("operatorType", "Unknown")
        arguments = plan.get("args", plan.get("arguments", {}))
        children = plan.get("children", [])
    else:
        operator = plan.operator_type
        arguments = plan.arguments
        children = plan.children

    print(f"{prefix}- Operator: {operator}")
    for key, value in arguments.items():
        print(f"{prefix}    {key}: {value}")

    for child in children:
        print_plan_tree(child, indent + 1)

def explain_query_plan(session: Session):
    result = session.run("""
        EXPLAIN
        MATCH (m:Movie {movieId: $id})
        RETURN m
        LIMIT 1
    """, id=8977)

    summary = result.consume()
    plan = summary.plan  # This may be a dict depending on driver version

    print("Logical Query Plan:\n" + "-"*40)
    print_plan_tree(plan)

explain_query_plan(session)

Logical Query Plan:
----------------------------------------
- Operator: ProduceResults@neo4j
    planner-impl: IDP
    string-representation: Cypher 5

Planner COST

Runtime PIPELINED

Runtime version 2025.05

Batch size 128

+------------------+----+-----------------+----------------+---------------------+
| Operator         | Id | Details         | Estimated Rows | Pipeline            |
+------------------+----+-----------------+----------------+---------------------+
| +ProduceResults  |  0 | m               |              1 |                     |
| |                +----+-----------------+----------------+                     |
| +Limit           |  1 | 1               |              1 |                     |
| |                +----+-----------------+----------------+                     |
| +Filter          |  2 | m.movieId = $id |              1 |                     |
| |                +----+-----------------+----------------+                     |
| +NodeByLabelScan |  3 | 

In [78]:
from typing import Union

def print_profile_tree(plan: Union[dict, object], indent=0):
    prefix = "  " * indent

    # Handle dict-style plan (fallback)
    if isinstance(plan, dict):
        operator = plan.get("operatorType", "Unknown")
        arguments = plan.get("args", plan.get("arguments", {}))
        children = plan.get("children", [])
        records = plan.get("records", "?")
        db_hits = plan.get("dbHits", "?")
    else:
        operator = plan.operator_type
        arguments = plan.arguments
        children = plan.children
        records = getattr(plan, "records", "?")
        db_hits = getattr(plan, "db_hits", "?")

    print(f"{prefix}- Operator: {operator}")
    for key, value in arguments.items():
        print(f"{prefix}    {key}: {value}")
    print(f"{prefix}    Rows: {records}")
    print(f"{prefix}    DB Hits: {db_hits}")

    for child in children:
        print_profile_tree(child, indent + 1)

def profile_query_plan(session: Session):
    result = session.run("""
        PROFILE
        MATCH (m:Movie {movieId: $id})
        RETURN m
        LIMIT 1
    """, id=123)

    summary = result.consume()
    plan = summary.profile

    print("Physical Execution Plan with Stats:\n" + "-"*50)
    print_profile_tree(plan)

profile_query_plan(session)

Physical Execution Plan with Stats:
--------------------------------------------------
- Operator: ProduceResults@neo4j
    GlobalMemory: 312
    planner-impl: IDP
    Memory: 0
    string-representation: Cypher 5

Planner COST

Runtime PIPELINED

Runtime version 2025.05

Batch size 128

+------------------+----+-----------------+----------------+------+---------+----------------+------------------------+-----------+---------------------+
| Operator         | Id | Details         | Estimated Rows | Rows | DB Hits | Memory (Bytes) | Page Cache Hits/Misses | Time (ms) | Pipeline            |
+------------------+----+-----------------+----------------+------+---------+----------------+------------------------+-----------+---------------------+
| +ProduceResults  |  0 | m               |              1 |    1 |      12 |              0 |                        |           |                     |
| |                +----+-----------------+----------------+------+---------+----------------+ 

### Benchmark:

- Prevent memory caching.
- Use indexed queries that simulate realistic application behavior.
- Measure read & writes.
- Randomized access patterns, multi-threading, or batch loads. (Out of scope)


In [18]:
# Create indexes: B-tree indexes (the default structure) in Neo4j
def create_indexes(session: Session) -> None:
    session.run("CREATE INDEX movie_id_index FOR (m:Movie) ON (m.movieId)")
    session.run("CREATE INDEX genres_index FOR (m:Movie) ON (m.genres)")
    session.run("CREATE INDEX user_id_index FOR (u:User) ON (u.userId)")

In [91]:
import random
from src.benchmark.utils import timed_op

@timed_op
def find_by_movie_id():
    movie_id = random.randint(1, 1000)
    try:
        with driver.session() as session:
            result = session.run("""
                MATCH (m:Movie {movieId: $movie_id})
                RETURN m
                LIMIT 1
            """, movie_id=movie_id)
            return [record["m"] for record in result]
    except Exception as e:
        print(f"[find_by_movie_id] Error with movieId={movie_id}: {e}")
        raise

@timed_op
def top_rated_movie():
    try:
        with driver.session() as session:
            result = session.run("""
                MATCH (m:Movie)
                WHERE m.avgRating > 4.5
                RETURN m
                LIMIT 10
            """)
            return [record["m"] for record in result]
    except Exception as e:
        print(f"[top_rated_movie] Error: {e}")
        raise

@timed_op
def search_by_genre():
    try:
        with driver.session() as session:
            result = session.run("""
                MATCH (m:Movie)
                WHERE "Comedy" IN m.genres
                RETURN m
                LIMIT 10
            """)
            return [record["m"] for record in result]
    except Exception as e:
        print(f"[search_by_genre] Error: {e}")
        raise

@timed_op
def search_by_title_regex():
    try:
        with driver.session() as session:
            result = session.run("""
                MATCH (m:Movie)
                WHERE toLower(m.title) CONTAINS "story"
                RETURN m
                LIMIT 5
            """)
            return [record["m"] for record in result]
    except Exception as e:
        print(f"[search_by_title_regex] Error: {e}")
        raise

@timed_op
def user_movie_join():
    user_ids = [random.randint(1, 1000) for _ in range(3)]
    try:
        with driver.session() as session:
            result = session.run("""
                MATCH (u:User)-[r:RATED]->(m:Movie)
                WHERE u.userId IN $user_ids
                RETURN u, collect(m) AS movies
                LIMIT 3
            """, user_ids=user_ids)
            return [{"user": record["u"], "movies": record["movies"]} for record in result]
    except Exception as e:
        print(f"[user_movie_join] Error with user_ids={user_ids}: {e}")
        raise

In [92]:
import statistics
import psutil
from src.benchmark.mongo_benchmark import BenchmarkResult
from typing import Callable


def run_benchmark(query_id: str,
                  run_query: Callable[[], tuple],
                  duration: int = 60,
                  warmup: int = 5) -> BenchmarkResult:
    sys_cpu_pct, sys_mem_pct, wall_ms = [], [], []
    success_count = 0
    errors = []

    psutil.cpu_percent(None)

    # Warmup
    for _ in range(warmup):
        try:
            run_query()
        except:
            pass

    t_end = time.time() + duration
    while time.time() < t_end:
        try:
            result, elapsed = run_query()
            wall_ms.append(elapsed / 1_000_000)
            success_count += 1
        except Exception as e:
            errors.append(str(e))

        sys_cpu_pct.append(psutil.cpu_percent(interval=None, percpu=False))
        sys_mem_pct.append(psutil.virtual_memory().percent)
        time.sleep(0.05)

    stats = {
        "query": query_id,
        "duration": duration,
        "iterations": len(wall_ms),
        "successes": success_count,
        "errors": len(errors),
        "wall_avg_ms": statistics.mean(wall_ms) if wall_ms else 0,
        "cpu_avg_pct": statistics.mean(sys_cpu_pct) if sys_cpu_pct else 0,
        "sys_mem_avg_pct": statistics.mean(sys_mem_pct) if sys_mem_pct else 0,
        "throughput_qps": len(wall_ms) / duration if duration > 0 else 0
    }

    return BenchmarkResult(
        summary=stats,
        series={
            "wall_ms": wall_ms,
            "sys_cpu_pct": sys_cpu_pct,
            "sys_mem_pct": sys_mem_pct,
        }
    )


In [94]:
queries = [
    ("find_by_id", find_by_movie_id),
    ("top_rated", top_rated_movie),
    ("genre_search", search_by_genre),
    ("title_search_regex", search_by_title_regex),
    ("user_ratings_with_movie_details", user_movie_join),
]

for query_id, query_func in queries:
    result = run_benchmark(query_id, query_func, duration=20)
    print(f"{query_id} result summary:", result.summary)

find_by_id result summary: {'query': 'find_by_id', 'duration': 20, 'iterations': 320, 'successes': 320, 'errors': 0, 'wall_avg_ms': 8.906941403125e-06, 'cpu_avg_pct': 7.1253125, 'sys_mem_avg_pct': 94.8575, 'throughput_qps': 16.0}
top_rated result summary: {'query': 'top_rated', 'duration': 20, 'iterations': 317, 'successes': 317, 'errors': 0, 'wall_avg_ms': 9.421760413249212e-06, 'cpu_avg_pct': 8.95173501577287, 'sys_mem_avg_pct': 94.8596214511041, 'throughput_qps': 15.85}
genre_search result summary: {'query': 'genre_search', 'duration': 20, 'iterations': 310, 'successes': 310, 'errors': 0, 'wall_avg_ms': 1.0807839032258064e-05, 'cpu_avg_pct': 7.287741935483871, 'sys_mem_avg_pct': 94.8016129032258, 'throughput_qps': 15.5}
title_search_regex result summary: {'query': 'title_search_regex', 'duration': 20, 'iterations': 321, 'successes': 321, 'errors': 0, 'wall_avg_ms': 8.634146937694704e-06, 'cpu_avg_pct': 6.667601246105919, 'sys_mem_avg_pct': 94.77881619937695, 'throughput_qps': 16.05}

In [ ]:
Explore
# Total de nodos por tipo
def contar_nodos():
    query = """
    MATCH (n)
    RETURN labels(n)[0] AS tipo, count(*) AS cantidad
    """
    return graph.run(query).to_table()

# Total de relaciones por tipo
def contar_relaciones():
    query = """
    MATCH ()-[r]->()
    RETURN type(r) AS relacion, count(*) AS cantidad
    """
    return graph.run(query).to_table()

# Usuarios sin ratings
def usuarios_sin_rating():
    query = """
    MATCH (u:User)
    WHERE NOT (u)-[:RATED]->()
    RETURN count(u) AS usuarios_sin_rating
    """
    return graph.run(query).to_table()

# Películas sin género ni rating ni genome tags
def peliculas_huerfanas():
    query = """
    MATCH (m:Movie)
    WHERE NOT (m)-[:HAS_GENRE]->()
      AND NOT (m)<-[:RATED]-()
      AND NOT (m)-[:HAS_RELEVANCE]->()
    RETURN count(m) AS peliculas_sin_relaciones
    """
    return graph.run(query).to_table()
print("Nodos por tipo:")
print(contar_nodos())

print("\nRelaciones por tipo:")
print(contar_relaciones())

print("\nUsuarios sin ratings:")
print(usuarios_sin_rating())

print("\nPelículas sin ningún vínculo:")
print(peliculas_huerfanas())